# 4.1  - Técnicas de preprocesado

## Carga de datos

### Dataset

Dataset URL: http://www.lsi.us.es/~fermin/corpusCine.zip

Nos quedamos con los ficheros .xml, que tienen una estructura del estilo:

```xml
<review author="XXX" title="XXX" rank="X" maxRank="X" source="XXX">
	<summary>XXX</summary>
	<body>XXX</body>
</review>
```

La codificación del texto es ISO-8859-15.

In [1]:
import os  # Utilerías del S.O.
import re  # Expresiones regulares

import config  # Ficheros de configuración

### Codificación

In [2]:
import chardet

with open(config.DATASET_MUCHOCINE_RAW+'/11.xml', 'r') as fd:
    txt = fd.read()
    
print chardet.detect(txt)

{'confidence': 0.8992448499992886, 'encoding': 'ISO-8859-2'}


### Con un parseador XML

Los ficheros paracen estar en formato XML.
La opción más razonable es ingerir los ficheros con un parseador XML.

Flujo de errores:
1. El parser no reconoce los documentos como XML.
    * Añadir la etiqueta `<?xml version="1.0" encoding="ISO-8859-15"?>`
1. Hay entidades XML no reconocidas (HTML). Ver http://www.degraeve.com/reference/specialcharacters.php
    * Añadir la etiqueta `<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">`.
    * Añadir entidades HTML al parser.
1. Hay documentos que no respetan la sintaxis porque incluyen el símbolo & en vez de la entidad &amp;
    * Este ya es un problema para las expresiones regulares.
    
    ![title](html/2problems-regex.jpg)

In [ ]:
import htmlentitydefs
import xml.etree.ElementTree as ET

def extract_from_document_parser(file_path):
    parser = ET.XMLParser()
    # Add HTLM entities to the parser
    for k,v in htmlentitydefs.name2codepoint.iteritems():
        parser.entity[k] = unichr(v)
    retval = {'file': os.path.basename(file_path)}
    with open(file_path,'r') as fd:
        f = fd.read()
    # Fixes solitary ampersands
    f = re.sub(' & ',' &amp; ', f)
    # Convert it into a XML file + entities definition
    parser.feed('<?xml version="1.0" encoding="ISO-8859-15"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">'+f)
    xml_root = parser.close()
    # Extract info
    for key in {'author', 'rank', 'title'}:
        retval[key] = xml_root.attrib.get('key')
    retval['summary'] = xml_root.find('summary').text
    retval['body'] = xml_root.find('body').text
    return retval

In [ ]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            extract_from_document_parser(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message
        
    

In [ ]:
with open(config.DATASET_MUCHOCINE_RAW+'/701.xml', 'r') as fd:
    f = fd.read()
f.split('\n')[2][1376:1379]

**Nota**: con [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) se puede procesar el dataset ya que no da errores. El propósito de esta sección es ser lo más agnóstico posible respecto a las herramientas y enseñar técnicas generales.

### Con expresiones regulares

Referencia de Python: https://docs.python.org/2/library/re.html#regular-expression-syntax

In [ ]:
import codecs  # Know your encoding

def extract_from_document_regexp(file_path):
    retval = {'file': os.path.basename(file_path)}
    with codecs.open(file_path,'r', encoding='ISO-8859-15') as fd:
        f = fd.read().strip()
    reg_expr = re.compile(r'\<review author="(?P<author>.*)" title="(?P<title>.*)" rank="(?P<rank>\d)".*\>\s*<summary>(?P<summary>.*)</summary>\s*<body>(?P<body>.*)</body>\s*</review>')
    regexp_result = reg_expr.search(f.strip())
    for key in ['author', 'rank', 'title', 'summary', 'body']:
        retval[key] = regexp_result.group(key)
    return retval

In [ ]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            extract_from_document_regexp(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message

## Alfabeto y conjunto de símbolos

In [ ]:
def filter_symbols(txt):
    ALPHABET = set(u'abcdefghijklmn\xf1opqrstuvwxyz0123456789 ')
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.lower().translate(table)
    txt = u''.join([x for x in txt if x in ALPHABET])
    return txt

In [ ]:
test_string1 = u'Deme un puré de patatas Martínez. ¡El mejor de Sigüenza!'
print filter_symbols(test_string1)

## Palabras comunes (stop words)

In [ ]:
## Taken from NLTK corpus so you don't have to download

#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#print filter_symbols(
#    ' '.join(
#        sorted(stopwords.words('spanish'))
#    )
#)

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

In [ ]:
test_string2 = u'Deme un puré de patatas Martínez. ¡El mejor de Sigüenza!'
print [
    word
    for word in filter_symbols(test_string2).split(' ')
    if word not in STOPWORDS
]

## Raíces léxicas

In [ ]:
from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

In [ ]:
test_string3 = u'Deme un puré de patatas Martínez. ¡El mejor de Sigüenza!'
print [
    stemmer.stem(word)
    for word in filter_symbols(test_string3).split(' ')
    if word not in STOPWORDS
]

## Todo junto

In [ ]:
for d in documents:
    for key in ['body', 'summary']:
        d[key+'_tokens'] = [
            stemmer.stem(word)
            for word in filter_symbols(d[key]).split(' ')
            if word not in STOPWORDS
        ]

In [ ]:
print documents[0]['summary']
print documents[0]['summary_tokens']

## Exportar/Importar

In [ ]:
import json
import zlib


# Keys to keep
keys_export = ['file', 'title', 'author','rank', 'summary_tokens', 'body_tokens']
# New data structure to export
docs_export = [
    {key: d[key] for key in keys_export}
    for d in documents
]

# Export
with open(config.DATASET_MUCHOCINE, 'w+') as fd:
    fd.write(zlib.compress(json.dumps(docs_export)))
    
# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    docs_import =json.loads(zlib.decompress(fd.read()))